In [6]:
pip install flask


  Using cached flask-3.1.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached werkzeug-3.1.3-py3-none-any.whl.metadata (3.7 kB)
  Using cached itsdangerous-2.2.0-py3-none-any.whl.metadata (1.9 kB)
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
Using cached flask-3.1.0-py3-none-any.whl (102 kB)
Using cached blinker-1.9.0-py3-none-any.whl (8.5 kB)
Using cached itsdangerous-2.2.0-py3-none-any.whl (16 kB)
Using cached werkzeug-3.1.3-py3-none-any.whl (224 kB)
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 2.3.7
    Uninstalling Werkzeug-2.3.7:
      Successfully uninstalled Werkzeug-2.3.7

[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
# 데이터 전처리 및 추천용 텍스트 준비
def prepare_recommendation_data(data):
    cleaned_data = []
    for item in data:
        # 카테고리와 타겟 결합
        combined_text = " ".join(item.get("category", []) + item.get("target", []))
        
        # 텍스트 전처리: 중복 제거 및 불필요한 단어 삭제
        combined_text = combined_text.replace("기타", "").replace("제한없음", "").strip()
        
        # 결과 저장
        item["combined_text"] = combined_text
        cleaned_data.append(item)
    return cleaned_data

# MongoDB에서 데이터 가져오기 (예제)
response = requests.get("http://127.0.0.1:5000/contest/list")
raw_data = response.json()

# 추천에 활용할 데이터 전처리
processed_data = prepare_recommendation_data(raw_data)

# 결과 확인
for item in processed_data:
    print(item["combined_text"])


광고/마케팅 논문/리포트 문학/글/시나리오 예체능/미술/음악 대외활동/서포터즈 일반인 대학생
기획/아이디어 광고/마케팅 네이밍/슬로건
기획/아이디어 광고/마케팅 대외활동/서포터즈 취업/창업  일반인 대학생
기획/아이디어 광고/마케팅 대외활동/서포터즈 취업/창업  일반인 대학생
광고/마케팅 영상/UCC/사진 웹/모바일/IT 예체능/미술/음악 대외활동/서포터즈
기획/아이디어 광고/마케팅 논문/리포트 네이밍/슬로건 대외활동/서포터즈
웹/모바일/IT 게임/소프트웨어 과학/공학 대외활동/서포터즈 일반인 대학생 청소년
기획/아이디어 광고/마케팅 대외활동/서포터즈 취업/창업  일반인 대학생
영상/UCC/사진 웹/모바일/IT 예체능/미술/음악  일반인 대학생
기획/아이디어 광고/마케팅 예체능/미술/음악 대외활동/서포터즈 봉사활동
기획/아이디어 광고/마케팅 대외활동/서포터즈 취업/창업  일반인 대학생
기획/아이디어 광고/마케팅 웹/모바일/IT 과학/공학 문학/글/시나리오 대학생
광고/마케팅 영상/UCC/사진 웹/모바일/IT 예체능/미술/음악 대외활동/서포터즈 봉사활동
대외활동/서포터즈 일반인 대학생
예체능/미술/음악 어린이
예체능/미술/음악
문학/글/시나리오 일반인 대학생
네이밍/슬로건
취업/창업
게임/소프트웨어
대외활동/서포터즈 대학생
봉사활동 일반인 대학생 청소년

대외활동/서포터즈
예체능/미술/음악 청소년
봉사활동 대학생
디자인/캐릭터/웹툰 대학생 청소년 어린이
영상/UCC/사진
대외활동/서포터즈 일반인 대학생
대외활동/서포터즈 일반인 대학생
기획/아이디어
예체능/미술/음악
대외활동/서포터즈 일반인 대학생
예체능/미술/음악
영상/UCC/사진
대학생
영상/UCC/사진
광고/마케팅 대학생
문학/글/시나리오
네이밍/슬로건
기획/아이디어
기획/아이디어
대외활동/서포터즈 일반인 대학생
예체능/미술/음악 일반인 대학생
봉사활동 일반인 대학생
영상/UCC/사진
예체능/미술/음악 대학생
대외활동/서포터즈 대학생
기획/아이디어
기획/아이디어 일반인 대학생
영상/UCC/사진
문학/글/시나

In [3]:
import requests
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 데이터 전처리 함수
def prepare_recommendation_data(data):
    cleaned_data = []
    for item in data:
        # 카테고리와 타겟 결합
        combined_text = " ".join(item.get("category", []) + item.get("target", []))
        
        # 텍스트 전처리: 중복 제거 및 불필요한 단어 삭제
        combined_text = combined_text.replace("기타", "").replace("제한없음", "").strip()
        
        # 결과 저장
        item["combined_text"] = combined_text
        cleaned_data.append(item)
    return cleaned_data

# TF-IDF 벡터화 및 코사인 유사도 계산
def calculate_similarity(data):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([item["combined_text"] for item in data])
    return cosine_similarity(tfidf_matrix)

# 유사 공모전 추천 함수
def get_similar_contests(contest_index, similarity_matrix, data, top_n=5):
    similarity_scores = list(enumerate(similarity_matrix[contest_index]))
    sorted_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    recommended_indices = [i for i, score in sorted_scores[1:top_n+1]]
    recommendations = [data[i] for i in recommended_indices]
    return recommendations

# MongoDB에서 데이터 가져오기 (예제)
response = requests.get("http://127.0.0.1:5000/contest/list")
raw_data = response.json()

# 추천에 활용할 데이터 전처리
processed_data = prepare_recommendation_data(raw_data)

# 유사도 계산
similarity_matrix = calculate_similarity(processed_data)

# 추천할 공모전 ID
contest_id = "5f3e6c90f7a3e0a9b6f19e91"  # 예시 공모전 ID
contest_index = next(i for i, item in enumerate(processed_data) if str(item["_id"]) == contest_id)

# 상위 5개 유사 공모전 추천
recommended_contests = get_similar_contests(contest_index, similarity_matrix, processed_data, top_n=5)

# 추천된 공모전 출력
for contest in recommended_contests:
    print(f"추천 공모전 ID: {contest['_id']}, 카테고리: {contest['category']}, 타겟: {contest['target']}")


StopIteration: 

In [4]:
for item in processed_data:
    print(item["_id"])  # _id 값을 출력



674b5f3ed4c246fb594a6e82
674b5f3ed4c246fb594a6e83
674b5f3ed4c246fb594a6e84
674b5f3ed4c246fb594a6e85
674b5f3ed4c246fb594a6e86
674b5f3ed4c246fb594a6e87
674b5f3ed4c246fb594a6e88
674b5f3ed4c246fb594a6e89
674b5f3ed4c246fb594a6e8a
674b5f3ed4c246fb594a6e8b
674b5f3ed4c246fb594a6e8c
674b5f3ed4c246fb594a6e8d
674b5f3ed4c246fb594a6e8e
674b5f3ed4c246fb594a6e8f
674b5f3ed4c246fb594a6e90
674b5f3ed4c246fb594a6e91
674b5f3ed4c246fb594a6e92
674b5f3ed4c246fb594a6e93
674b5f3ed4c246fb594a6e94
674b5f3ed4c246fb594a6e95
674b5f3ed4c246fb594a6e96
674b5f3ed4c246fb594a6e97
674b5f3ed4c246fb594a6e98
674b5f3ed4c246fb594a6e99
674b5f3ed4c246fb594a6e9a
674b5f3ed4c246fb594a6e9b
674b5f3ed4c246fb594a6e9c
674b5f3ed4c246fb594a6e9d
674b5f3ed4c246fb594a6e9e
674b5f3ed4c246fb594a6e9f
674b5f3ed4c246fb594a6ea0
674b5f3ed4c246fb594a6ea1
674b5f3ed4c246fb594a6ea2
674b5f3ed4c246fb594a6ea3
674b5f3ed4c246fb594a6ea4
674b5f3ed4c246fb594a6ea5
674b5f3ed4c246fb594a6ea6
674b5f3ed4c246fb594a6ea7
674b5f3ed4c246fb594a6ea8
674b5f3ed4c246fb594a6ea9


In [5]:
contest_index = next((i for i, item in enumerate(processed_data) if str(item["_id"]) == contest_id), None)


In [6]:
if contest_index is None:
    print(f"공모전 ID {contest_id}를 찾을 수 없습니다.")
else:
    # 상위 5개 유사 공모전 추천
    recommended_contests = get_similar_contests(contest_index, similarity_matrix, processed_data, top_n=5)
    for contest in recommended_contests:
        print(f"추천 공모전 ID: {contest['_id']}, 카테고리: {contest['category']}, 타겟: {contest['target']}")


공모전 ID 5f3e6c90f7a3e0a9b6f19e91를 찾을 수 없습니다.


In [7]:
from bson import ObjectId

# contest_id가 ObjectId 형식이라면
contest_id = ObjectId("5f3e6c90f7a3e0a9b6f19e91")

contest_index = next((i for i, item in enumerate(processed_data) if item["_id"] == contest_id), None)


In [8]:
from bson import ObjectId

# contest_id가 ObjectId 형식이라면
contest_id = ObjectId("674b5f3ed4c246fb594a6eb7")  # 예시로 출력된 _id 값 사용

# contest_id에 해당하는 공모전 인덱스 찾기
contest_index = next((i for i, item in enumerate(processed_data) if item["_id"] == contest_id), None)

# contest_index가 None이 아닌 경우 추천을 진행
if contest_index is None:
    print(f"공모전 ID {contest_id}를 찾을 수 없습니다.")
else:
    # 상위 5개 유사 공모전 추천
    recommended_contests = get_similar_contests(contest_index, similarity_matrix, processed_data, top_n=5)
    for contest in recommended_contests:
        print(f"추천 공모전 ID: {contest['_id']}, 카테고리: {contest['category']}, 타겟: {contest['target']}")


공모전 ID 674b5f3ed4c246fb594a6eb7를 찾을 수 없습니다.


In [9]:
for item in processed_data:
    print(type(item["_id"]), item["_id"])  # _id 값의 타입과 값을 출력


<class 'str'> 674b5f3ed4c246fb594a6e82
<class 'str'> 674b5f3ed4c246fb594a6e83
<class 'str'> 674b5f3ed4c246fb594a6e84
<class 'str'> 674b5f3ed4c246fb594a6e85
<class 'str'> 674b5f3ed4c246fb594a6e86
<class 'str'> 674b5f3ed4c246fb594a6e87
<class 'str'> 674b5f3ed4c246fb594a6e88
<class 'str'> 674b5f3ed4c246fb594a6e89
<class 'str'> 674b5f3ed4c246fb594a6e8a
<class 'str'> 674b5f3ed4c246fb594a6e8b
<class 'str'> 674b5f3ed4c246fb594a6e8c
<class 'str'> 674b5f3ed4c246fb594a6e8d
<class 'str'> 674b5f3ed4c246fb594a6e8e
<class 'str'> 674b5f3ed4c246fb594a6e8f
<class 'str'> 674b5f3ed4c246fb594a6e90
<class 'str'> 674b5f3ed4c246fb594a6e91
<class 'str'> 674b5f3ed4c246fb594a6e92
<class 'str'> 674b5f3ed4c246fb594a6e93
<class 'str'> 674b5f3ed4c246fb594a6e94
<class 'str'> 674b5f3ed4c246fb594a6e95
<class 'str'> 674b5f3ed4c246fb594a6e96
<class 'str'> 674b5f3ed4c246fb594a6e97
<class 'str'> 674b5f3ed4c246fb594a6e98
<class 'str'> 674b5f3ed4c246fb594a6e99
<class 'str'> 674b5f3ed4c246fb594a6e9a
<class 'str'> 674b5f3ed4c

In [10]:
contest_id = "674b5f3ed4c246fb594a6eb7"  # 예시로 출력된 _id 값

# contest_id에 해당하는 공모전 인덱스 찾기
contest_index = next((i for i, item in enumerate(processed_data) if str(item["_id"]) == contest_id), None)


In [11]:
contest_id = "674b5f3ed4c246fb594a6ee3"  # 예시로 출력된 _id 값

# contest_id에 해당하는 공모전 인덱스 찾기
contest_index = next((i for i, item in enumerate(processed_data) if item["_id"] == contest_id), None)

# 해당 contest_id가 없으면 None이 반환됨
if contest_index is None:
    print(f"공모전 ID {contest_id}를 찾을 수 없습니다.")
else:
    print(f"공모전 ID {contest_id}를 찾았습니다. 인덱스: {contest_index}")


공모전 ID 674b5f3ed4c246fb594a6ee3를 찾았습니다. 인덱스: 97


In [17]:
import requests
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 데이터 전처리 및 추천용 텍스트 준비
def prepare_recommendation_data(data):
    cleaned_data = []
    for item in data:
        # 카테고리와 타겟 결합
        combined_text = " ".join(item.get("category", []) + item.get("target", []))
        
        # 텍스트 전처리: 중복 제거 및 불필요한 단어 삭제
        combined_text = combined_text.replace("기타", "").replace("제한없음", "").strip()
        
        # 결과 저장
        item["combined_text"] = combined_text
        cleaned_data.append(item)
    return cleaned_data

# 유사 공모전 추천 함수
def get_similar_contests(contest_index, similarity_matrix, data, top_n=5):
    similarity_scores = list(enumerate(similarity_matrix[contest_index]))
    sorted_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    recommended_contests = [data[i] for i, score in sorted_scores[1:top_n+1]]  # 자기 자신 제외
    return recommended_contests

# MongoDB에서 데이터 가져오기 (예제)
response = requests.get("http://127.0.0.1:5000/contest/list")
raw_data = response.json()

# 추천에 활용할 데이터 전처리
processed_data = prepare_recommendation_data(raw_data)

# 추천할 공모전 ID
contest_id = "674b5f3ed4c246fb594a6e84"  # 예시 공모전 ID

# contest_id에 해당하는 공모전 인덱스 찾기
contest_index = next((i for i, item in enumerate(processed_data) if item["_id"] == contest_id), None)

# 해당 contest_id가 없으면 None이 반환됨
if contest_index is None:
    print(f"공모전 ID {contest_id}를 찾을 수 없습니다.")
else:
    print(f"공모전 ID {contest_id}를 찾았습니다. 인덱스: {contest_index}")

    # TF-IDF 벡터화
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([item["combined_text"] for item in processed_data])

    # 코사인 유사도 계산
    similarity_matrix = cosine_similarity(tfidf_matrix)

    # 상위 5개 유사 공모전 추천
    recommended_contests = get_similar_contests(contest_index, similarity_matrix, processed_data, top_n=5)

    # 추천 결과 출력
    print("추천 공모전:")
    for contest in recommended_contests:
        print(f"공모전 ID: {contest['_id']}, 제목: {contest['title']}")


공모전 ID 674b5f3ed4c246fb594a6e84를 찾았습니다. 인덱스: 2
추천 공모전:
공모전 ID: 674b5f3ed4c246fb594a6e85, 제목: [무료] 삼성생명 금융 현직자 멘토링 12월 주말반 모집
공모전 ID: 674b5f3ed4c246fb594a6e89, 제목: [무료] 2025 삼성 Financial Networks 금융연수프로그램 7기 모집
공모전 ID: 674b5f3ed4c246fb594a6e8c, 제목: [CJ CGV(주)/(주)쇼박스/(주)MBC플러스]영화/드라마등 콘텐츠 과몰입러가 90일만에 대기업 2곳에서 '1+1'  직무스펙 인정받고 취업하는 가장 빠른 방법! (~12/01 일요일)
공모전 ID: 674b5f3ed4c246fb594a6e94, 제목: 2024년 이노베이션스퀘어확산사업 창업경진대회
공모전 ID: 674b5f3ed4c246fb594a6eb3, 제목: 2024 스마트팜 인공지능활용 아이디어 챌린지


유사한 공모전 추천 시스템에서 **유사성**을 정의하는 기준은 두 가지 주요 요소로 나눠볼 수 있습니다:

### 1. **카테고리와 타겟 정보의 결합**
   - 추천 시스템에서는 각 공모전의 **`category`** (카테고리)와 **`target`** (타겟)을 결합하여 **텍스트 데이터**를 생성합니다.
   - 예를 들어, "광고/마케팅"과 "대학생"을 타겟으로 하는 공모전은 그 텍스트 데이터를 기반으로 유사도를 계산하게 됩니다.
   - 이 텍스트는 각 공모전의 주요 특징을 대표하는 문자열로, 다른 공모전과 비교될 때 유사성을 파악하는 중요한 기준이 됩니다.

### 2. **TF-IDF (Term Frequency-Inverse Document Frequency) 벡터화**
   - **TF-IDF**는 각 텍스트 내 단어의 중요도를 평가하는 방법입니다. 이 방법을 사용하여 공모전의 `combined_text`를 수치화된 벡터로 변환합니다.
     - **TF (Term Frequency)**: 특정 단어가 텍스트 내에서 얼마나 자주 등장하는지
     - **IDF (Inverse Document Frequency)**: 해당 단어가 얼마나 희귀한지
   - 이렇게 변환된 벡터는 각 공모전의 특징을 수치적으로 표현하며, 이를 통해 각 공모전 간의 **유사도**를 계산할 수 있습니다.

### 3. **코사인 유사도 (Cosine Similarity)**
   - **코사인 유사도**는 두 벡터 간의 **각도**를 측정하여 얼마나 유사한지 판단하는 방법입니다.
     - 코사인 유사도는 벡터 간의 내적을 이용해 계산되며, 값은 0에서 1 사이입니다. 값이 1에 가까울수록 두 벡터(즉, 두 공모전)는 더 유사하다고 간주됩니다.
     - 예를 들어, 두 공모전이 비슷한 **카테고리**와 **타겟**을 가질 경우, 그들의 `combined_text` 벡터는 비슷한 형태를 가지게 되며, 코사인 유사도 값이 높게 나옵니다.

### 4. **추천 시스템의 유사성 계산 과정**
   - 사용자가 요청한 공모전(`contest_id`)을 기준으로 **`combined_text`** (카테고리 + 타겟 정보 결합)를 사용하여 **TF-IDF 벡터화**를 진행합니다.
   - 그런 후, 각 공모전 간의 **코사인 유사도**를 계산하여, 해당 공모전과 가장 비슷한 공모전을 추천합니다.
   - **유사도 값이 높은 공모전들**이 추천 리스트에 포함됩니다.

### 예시:

- 공모전 1: 
  - **category**: "광고/마케팅"
  - **target**: "대학생"
  - **combined_text**: "광고/마케팅 대학생"
  
- 공모전 2:
  - **category**: "광고/마케팅"
  - **target**: "일반인"
  - **combined_text**: "광고/마케팅 일반인"
  
- 공모전 3:
  - **category**: "영상/UCC/사진"
  - **target**: "대학생"
  - **combined_text**: "영상/UCC/사진 대학생"

이 경우, **공모전 1**과 **공모전 2**는 비슷한 **카테고리**(광고/마케팅)를 가지고 있지만, **타겟**이 다르므로, **공모전 3**과는 더 큰 차이가 있을 것입니다. 따라서, **공모전 1과 공모전 2**가 더 유사하다고 판단될 수 있습니다.

### 유사성의 기준을 결정하는 방법:
- **카테고리**와 **타겟**이 중요한 요소로 작용하며, 이 정보들이 결합된 후 **코사인 유사도**를 기반으로 추천을 진행합니다. 
- 추천 시스템에서 유사성은 **텍스트**를 기반으로 한 벡터 차원에서 계산되므로, **카테고리**와 **타겟**의 내용이 유사한 공모전일수록 추천 결과로 나올 확률이 높습니다.

이와 같은 방식으로 공모전 간의 유사도를 계산하여, 사용자가 관심을 가질만한 **상위 5개의 유사 공모전**을 추천할 수 있습니다.